## COCO     
COCO数据集(Common objects in Context)，是微软团队发起的一个可以用来作图像识别+分割+标题的图像数据集，官方网址：http://cocodataset.org 。       

官方除了数据以外, 也提供了一组API(一般都称之为COCO API)来协助加载，解析和可视化COCO中的标注(annotations)。 API支持对象实例(object instance)，对象关键点(object keypoints)和图像标题(image caption)标注(annotation)资讯的提取。


In [0]:
from pycocotools.coco import COCO
import os
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab
pylab.rcParams['figure.figsize'] = (10.0, 8.0)

In [0]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip
!wget http://images.cocodataset.org/zips/train2014.zip 

--2019-06-30 10:23:05--  http://images.cocodataset.org/annotations/annotations_trainval2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.216.0.128
Connecting to images.cocodataset.org (images.cocodataset.org)|52.216.0.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 252872794 (241M) [application/zip]
Saving to: ‘annotations_trainval2014.zip’

annotations_trainva 100%[===================>] 241.16M  96.5MB/s    in 2.5s    

2019-06-30 10:23:08 (96.5 MB/s) - ‘annotations_trainval2014.zip’ saved [252872794/252872794]

--2019-06-30 10:23:09--  http://images.cocodataset.org/zips/train2014.zip
Resolving images.cocodataset.org (images.cocodataset.org)... 52.217.0.59
Connecting to images.cocodataset.org (images.cocodataset.org)|52.217.0.59|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13510573713 (13G) [application/zip]
Saving to: ‘train2014.zip’

train2014.zip       100%[===================>]  12.58G  61.7MB/s    in 

In [0]:
!unzip annotations_trainval2014.zip -d /content/coco/
!unzip train2014.zip -d /content/coco/

Output hidden; open in https://colab.research.google.com to view.

In [0]:
!ls

sample_data


In [0]:
root = os.getcwd()

COCO_DATA_PATH = os.path.join(root, "coco")

NameError: ignored

In [0]:
dataDir = COCO_DATA_PATH
dataType = 'train2014'

annFilename = 'instances_%s.json'%(dataType) 
annFile = os.path.join(dataDir,"annotations",annFilename) 

In [0]:
print(annFilename)
print(annFile)

instances_train2014.json
/content/coco/annotations/instances_train2014.json


## 1. COCO数据集标注    
* 对象实例 `object instance`
* 对象关键点 `object keypoints`
* 图像标注 `image caption`

### 1.1 `object instance` 对象实例   
每个实例标注(object instance)包含一系列的`key:value`的栏位，包括对象的类别ID和图像分割遮罩(segment mask)。      

分割(segment)格式取决于实例是否代表单个对象（iscrowd = 0，在这种情况下使用多边形）或对象集合（iscrowd = 1，在这种情况下使用RLE）。请注意，单个对象（iscrowd = 0）可能需要多个多边形，例如，如果被遮挡。人群注释（iscrowd = 1）用于标记大量对象（例如一群人）。    
此外，还为每个对象提供了一个封闭的边界框（框坐标是从左上角的图像角度测量的，并且是0索引的）。最后，注解结构的类别字段存储了类别标识到类别和超类别名称的映射。

```
annotation{
    "id" : int, 
    "image_id" : int, 
    "category_id" : int, 
    "segmentation" : RLE or [polygon], 
    "area" : float, 
    "bbox" : [x,y,width,height], 
    "iscrowd" : 0 or 1,
}

categories[{
    "id" : int, 
    "name" : str, 
    "supercategory" : str,
}]
```

注解: 游程编码（RLE，run-length encoding），又称行程长度编码或变动长度编码法，是一种与资料性质无关的无损数据压缩技术

In [0]:
# 初始化COCO API并载入对象实例的标注文件
coco=COCO(annFile)

loading annotations into memory...
Done (t=12.94s)
creating index...
index created!


## 2. COCO API   
为了方便取得在COCO资料集里图像与各种不同标注(annotations)的资讯， COCO提供了以下API方便用来提取相关资讯:  
*  `download` 从mscoco.org服务器下载COCO图像
* `getAnnIds` 获取满足给定过滤器条件的annotation ids
* `getCatIds` 获取满足给定过滤条件的category ids 
* `getImgIds` 获取满足给定过滤条件的imgage ids 
* `loadAnns` 用指定的id加载annotations 
* `loadCats` 使用指定的id加载categories 
* `loadImgs` 用指定的id加载imgs 
* `loadRes` 加载算法结果并创建用于访问它们的API 
* `showAnns` 显示指定的标注资讯


### 2.1 显示COCO类别和超类别      
```
categories[{
    "id" : int, 
    "name" : str, 
    "supercategory" : str,
}]
```

In [0]:
cats = coco